In [1]:
#from pip._internal import main
#main(["install", "msgpack"])
#main(["install", "pixiedust"])

In [1]:
import requests, bs4, selenium, pickle#, inspect, pixiedust
from selenium import webdriver
conj = ["#cat=", "&subcat=", "&spec="]
lev = ["topic", "subtopic", "specialty"]

In [2]:
def climbTree(dic, link, level, ls, topic):
    if not dic:
        ls.update({link : topic})
    else:
        for term in dic:
            nlink = link + conj[level]+ term.lower().replace(" ", "").replace("'", "")
            ls = climbTree(dic[term], nlink, level+1, ls, topic+"/"+term)
    return ls


In [3]:
class CourseRes:
    formato=""
    typeR=""
    nome=""
    link=""
    
    def __init__(self, nome, formato, typeR, link):
        self.nome=nome
        self.formato=formato
        self.typeR=typeR
        self.link=link
        
    def toString(self):
        return "{} {}.{} in {}".format(self.typeR, self.nome, self.formato, self.link)

class CourseInfo:
    nome=""
    idN=""
    link=""
    kywTit=""
    difficulty=""
    res=[] # type CourseRes
    durata=[]
    min_age=0
    max_age=0
    desc=""
    discip=""
    pubblico=[]
    
    def __init__(self, link, difficulty):
        self.nome=nome
        self.difficulty=difficulty
        self.link=link
        if "undergraduate" in difficulty.lower():
            self.pubblico = ['Middle School', 'High School']
            self.min_age=11
            self.max_age=18
        else:
            self.pubblico = ['College General Ed', 'Graduate School', 'Professional']
            self.min_age=19
            self.max_age=100
            
    
    #difficulty: graduate=medio alta, undergraduate=media
    #formato: lecture notes=pdf, assignments=pdf, 
    #durata: slidex2 = minuti {[0-30]...[120+]}, 
    #pubblico: undergrad=['Middle School', 'High School'], graduate=['College General Ed', 'Graduate School', 'Professional]
    #type: lecture notes=presentation, assignments=Quiz/Test, 
    #min_age: undergrad=11, grad=19, max_age: undergrad=18, grad=100
        
    def printInCSV(self, file):
        for attr in self.__dict__:
            item = self.__dict__[attr]
            if not type(item) is list:
                file.write("{};".format(item))
            else:
                if isinstance(item[0], CourseRes):
                    file.write("{};".format([r.toString() for r in item]))
        file.write("\n")

In [11]:
url = 'https://ocw.mit.edu/courses/find-by-topic/'
PATH = './driver/chromedriver.exe'
driver = webdriver.Chrome(PATH)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36'
}

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_18036/2805806334.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [5]:
data = pickle.load(open('MIT_ontology_DAG.pickle', 'rb'))
courseList = climbTree(data, url, 0, {}, "")

In [6]:
pref_url = "https://ocw.mit.edu"
for link in courseList:
    driver.get(link)
    html = driver.page_source
    for s in bs4.BeautifulSoup(html, "html.parser").find_all("div", {"class": "courseListDiv"}):
        for s1 in s.find_all("li", {"class" : "courseTitleCol"}):
            for s2 in s1.find_all("a"):
                objCourse= CourseInfo(s2.text, courseList[link], pref_url+s2.attrs["href"])
                #collectDataInObj(objCourse)  #Todo
                objCourse.printInCSV(open("res_dataset.csv", "a"))
                del objCourse

In [13]:
#obj = CourseInfo({})
#obj.res=[CourseRes("prova", "pdf", "presentazione", "culo a tua mamma")]
#obj.nome="stocazzo"
#f = open("demofile2.csv", "a")
#obj.printInCSV(f)
#f.close()
#f1 = open("demofile2.csv", "rb")
#for line in f1:
#    print(line)